In [7]:
from pathlib import Path
import os
import hjson
import numpy as np
import pandas as pd
from IPython.display import display
from loguru import logger
import time

# SolarMED modeling
from solarmed_modeling.utils.matlab_environment import set_matlab_environment

# Visualization
from phd_visualizations.test_timeseries import experimental_results_plot
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

# Setup environment for running MATLAB code if not done externally
# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"

set_matlab_environment()
logger.enable("solarmed_modeling")

# auto reload modules
%load_ext autoreload
%autoreload 2

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'
config_path: Path = Path('../data')
output_path: Path = Path('../docs/attachments')

# filename_opt_result = '20240108_optimization_results.json'
# Debería ser un .csv al que se le hayan añadido las variables faltantes desde librescada:
# - J de variadores y medidor de potencia
# - FT-DES-002_VFD

date_str: str = '20230703'

filename_process_data = f'{date_str}_solarMED.csv'
# filename_process_data = '20230505_solarMED.csv'

filename_process_data2 = f'{date_str}_MED.csv'

# Resample figures using plotly_resampler
resample_figures = False

sample_rate = '60s'
# sample_rate = '300s'
sample_rate_numeric = int(sample_rate[:-1])

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity

# initial_datetime = '2024-01-08 10:55'
# final_datetime = '2024-01-08 14:00'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Pre-processing

In [8]:
from solarmed_modeling.utils import data_preprocessing, data_conditioning

data_paths = [data_path / filename_process_data, data_path / filename_process_data2]

with open( config_path / "variables_config.hjson") as f:
    vars_config = hjson.load(f)
    
# Load data and preprocess data
df = data_preprocessing(data_paths, vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(df, cost_w=cost_w, cost_e=cost_e, sample_rate_numeric=sample_rate_numeric)

2024-09-19 16:07:20.734 | INFO     | solarmed_modeling.utils:data_preprocessing:112 - Reading data from 20230703_solarMED.csv
2024-09-19 16:07:21.018 | DEBUG    | solarmed_modeling.utils:process_dataframe:68 - Index([], dtype='object')
2024-09-19 16:07:21.020 | INFO     | solarmed_modeling.utils:process_dataframe:75 - Number of duplicate index values in df: 0
2024-09-19 16:07:21.054 | INFO     | solarmed_modeling.utils:data_preprocessing:132 - Reading data from 20230703_MED.csv
2024-09-19 16:07:21.355 | DEBUG    | solarmed_modeling.utils:process_dataframe:68 - Index([], dtype='object')
2024-09-19 16:07:21.357 | INFO     | solarmed_modeling.utils:process_dataframe:75 - Number of duplicate index values in df: 0
2024-09-19 16:07:21.361 | DEBUG    | solarmed_modeling.utils:data_preprocessing:151 - Common columns in both dataframes: Index(['FT-AQU-100', 'TT-AQU-107a', 'CT-DES-001', 'FT-DES-003', 'FT-DES-005',
       'SW2TC1', 'TE-DES-015', 'TT-DES-030'],
      dtype='object'), dropping them

## Visualize test data

In [3]:
# df.to_csv('data/dev.csv')

In [ ]:
# Update plot config
with open( config_path / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)

In [ ]:
# Test model
from solarmed_modeling.solar_med import SolarMED

idx_start = 1
span = 1
idx_end = len(df)
df_mod = pd.DataFrame()

# Initialize model
model = SolarMED(
    resolution_mode='simple',
    use_models=True,
    use_finite_state_machine=True,
    
    sample_time=sample_rate_numeric,
    
    # If a slow sample time is used, the solar field internal PID needs to be detuned
    # Ki_sf=-0.0001,
    # Kp_sf=-0.005,
    
    # Initial states
    ## Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    
    ## Solar field
    Tsf_in_ant=df['Tsf_in'].iloc[idx_start-span:idx_start].values,
    msf_ant=df['qsf'].iloc[idx_start-span:idx_start].values,
    
    # cost_w = 3, # €/m³ 
    # cost_e = 0.05, # €/kWhe,
)

# Save model initial state and configuration
model_config = model.model_dump_configuration()
df_mod = model.to_dataframe(df_mod)

model_dump = model.model_dump()

# Run model
# %autoreload 2

for idx in range(idx_start, idx_end):
    # idx = 1
    ds = df.iloc[idx]
    
    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
        
    model.step(
        # Decision variables
        ## MED
        mmed_s=ds['qmed_s'],
        mmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        mts_src=ds['qhx_s'],
        ## Solar field
        Tsf_out=ds['Tsf_out'],
        
        med_vacuum_state=2,
        
        # Inputs
        # When the solar field is starting up, a flow can be provided to sync the model with the real system, if a valid Tsf_out is provided, it will be prioritized
        msf=ds['qsf'] if ds['qsf'] > 4 else None,
        
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f"Finished Iteration {idx} / {idx_end} - {df.index[idx]:%H:%M:%S}, elapsed time: {time.time()-start_time:.2f} seconds.")

    df_mod = model.to_dataframe(df_mod)

2024-09-19 16:07:50.164 | INFO     | solarmed_modeling.solar_med:init_matlab_engine:717 - MATLAB engine initialized
2024-09-19 16:07:50.169 | INFO     | solarmed_modeling.solar_med:model_post_init:538 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: simple
            - Sample time: 60.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
        
2024-09-19 16:07:50.346 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:47 - (mmed_s_sp) Value 0.03 is less than 30.00 -> 0.0
2024-09-19 16:07:50.347 | DEBUG    | solarmed_modeling.data_validation:within_range_or_zero_or_max:47 - (mmed_f_sp) Value 0.85 is less than 5.00 -> 0.0
2024-09-19 16:07:50.347 | DEBUG    | sol

In [6]:
# Sync model index with measured data
df_mod.index = df.index[idx_start-1:idx if idx<idx_end-1 else idx_end] # idx_start-1 because now we are adding one element after the initialization

# Update plot config
with open( config_path / "plot_config_validation.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)

In [ ]:
# Save figure
save_figure(
    figure_name=f"SolarMED_validation_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=output_path,
    fig=fig, formats=('png', 'html'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)

In [ ]:
from solarMED_optimization.visualization import plot_episode_state_evolution
from solarmed_modeling import SF_TS_State, MedState

fig = plot_episode_state_evolution(df_mod, subsystems_state_cls=[SF_TS_State, MedState], show_edges=False)

fig

In [ ]:
# Using PIL, stack images
from PIL import Image

def stack_images_vertically(image_path_top: Path, image_path_bottom: Path, output_path: Path):
    # Load the images
    top_image = Image.open(image_path_top)
    bottom_image = Image.open(image_path_bottom)
    
    # Calculate dimensions for the new image
    total_height = top_image.height + bottom_image.height
    max_width = max(top_image.width, bottom_image.width)
    
    # Create a new image with appropriate dimensions
    new_image = Image.new('RGB', (max_width, total_height))
    
    # Paste the top image at the top of the new image
    new_image.paste(top_image, (0, 0))
    
    # Paste the bottom image below the top image
    new_image.paste(bottom_image, (0, top_image.height))
    
    # Save or display the new image
    new_image.save(output_path, )
    # new_image.show()  # Uncomment to display the image

In [ ]:
# Hihglight state evolution to generate a GIF
from cairosvg import svg2png
from copy import deepcopy
from lxml import etree
from solarMED_optimization.visualization.operating_mode import SolarMEDState

src_diagram_path = Path('//home/patomareao/development_psa/SolarMED-optimization/Material auxiliar/solarMED_optimization-Operating modes.svg')

gif_path: Path = Path("/home/patomareao/Documents/gif_attempt")
plot_path = gif_path / "plot"
plot_path.mkdir(exist_ok=True)

diagram_path = gif_path / "diagram"
diagram_path.mkdir(exist_ok=True)

height=600
width=1800


with open(src_diagram_path, 'r') as f:
    diagram_file = etree.parse(f)

fig = plot_episode_state_evolution(df_mod, subsystems_state_cls=[SF_TS_State, MedState], show_edges=False)

for step_idx in fig.layout.xaxis.tickvals:
    # Generate plot
    plot_episode_state_evolution(df_mod, subsystems_state_cls=[SF_TS_State, MedState], 
                                 show_edges=False, highligth_step=step_idx,
                                 width=width, height=height).write_image(plot_path / f"step_{step_idx:03}.png")
    
    # Generate diagram
    sf_state, ts_state, med_state = df_mod.loc[df_mod.index[step_idx], ['sf_state', 'ts_state', 'med_state']]
    solar_med_state = SolarMEDState(sf_state=sf_state.value, ts_state=ts_state.value, med_state=med_state.value)

    diagram = solar_med_state.create_state_diagram(deepcopy(diagram_file))
    diagram.write( diagram_path / f'step_{step_idx:03}.svg')
    
    svg2png(url=str(diagram_path / f'step_{step_idx:03}.svg'), write_to=str(diagram_path / f'step_{step_idx:03}.png'), 
            output_width=width, output_height=height, dpi=300, background_color='white')

    stack_images_vertically(plot_path / f"step_{step_idx:03}.png", diagram_path / f"step_{step_idx:03}.png", gif_path / f"step_{step_idx:03}.png")


In [ ]:
# Generate GIF
# Call ffmpeg from the command line

## BETTER:
# magick *.png -morph 3 -delay 50 -loop 0 output.gif
# ffmpeg -framerate 3 -pattern_type glob -i '*.png' output.gif

In [ ]:
from IPython.display import SVG
from lxml import etree
from solarMED_optimization.visualization.operating_mode import SolarMEDState, SolarFieldState, ThermalStorageState, MEDState
from copy import deepcopy

diagram_path = Path('//home/patomareao/development_psa/SolarMED-optimization/Material auxiliar/solarMED_optimization-Operating modes.svg')

with open(diagram_path, 'r') as f:
    diagram_file = etree.parse(f)

solar_med_state = SolarMEDState(sf_state=, ts_state=ThermalStorageState.ACTIVE, med_state=MEDState.SHUTTING_DOWN)

diagram = solar_med_state.create_state_diagram(deepcopy(diagram_file))

SVG(etree.tostring(diagram))

In [ ]:
sf_state, ts_state, med_state = df_mod.loc[df_mod.index[step_idx], ['sf_state', 'ts_state', 'med_state']]

SolarFieldState(sf_state.value)


In [ ]:
# from solarmed_modeling import SF_TS_State, MedState
# from solarMED_optimization.path_exploration import Node, generate_edges, generate_edges_dataframe
# from solarMED_optimization.visualization import plot_state_graph
# 
# Np = len(df_mod)
# 
# # Generate nodes dataframes
# nodes_sfts_df = pd.DataFrame([
#     Node(step_idx=step_idx,state=state).model_dump()
#     for step_idx in range(Np) for state in [state for state in SF_TS_State]
# ])
# nodes_med_df = pd.DataFrame([
#     Node(step_idx=step_idx, state=state).model_dump()
#     for step_idx in range(Np) for state in [state for state in MedState]
# ])
# 
# # Generate edges dataframes
# edges_list_med = []; edges_list_sfts = []
# for step_idx in range(Np):
#     edges_list_med = generate_edges(edges_list_med, step_idx, system='MED', Np=Np)
#     edges_list_sfts = generate_edges(edges_list_sfts, step_idx, system='SFTS', Np=Np)
#     
# edges_df_med = generate_edges_dataframe(edges_list_med, nodes_df=nodes_med_df)
# edges_df_sfts = generate_edges_dataframe(edges_list_sfts, nodes_df=nodes_sfts_df)
# 
# 
# fig = plot_state_graph(
#     nodes_df=[nodes_sfts_df, nodes_med_df], 
#     system='SolarMED',
#     # edges_df=[edges_df_sfts, edges_df_med],
#     results_df=df_mod,
#     Np=Np, 
#     height=800,
#     width=1200,
# )
# 
# fig

In [ ]:
# Save figure
save_figure(
    figure_name=f"SolarMED_state_evolution_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=output_path,
    fig=fig, formats=('png', 'html', 'svg'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)